# 🎨 Notebook 01: Train Transforms — Data Augmentation Pipeline

**Purpose:** Build a robust augmentation pipeline for training data to improve model generalization.

**What you'll learn:** How to compose transforms in the correct order, why augmentation matters, and how normalization stabilizes training.


## 🎯 Concept Primer: Why Data Augmentation?

### The Problem: Limited Data
- Deep learning models need **thousands** of examples to generalize well
- Medical imaging datasets are expensive to label (expert pathologists needed)
- Small datasets → **overfitting** (model memorizes training examples)

### The Solution: Data Augmentation
- Create **synthetic variety** by applying realistic transformations
- Horizontal flips, rotations, color jitter → model learns invariant features
- **Only applied to training data** (val/test need consistency)

### Transform Order Matters!
```
✅ CORRECT:
Resize → Augmentation (Flip, Rotate, ColorJitter) → ToTensor → Normalize

❌ WRONG:
ToTensor → ColorJitter  (ColorJitter expects PIL images, not tensors!)
Normalize → Resize  (Normalize expects [0,1] tensor values)
```

### Normalization Deep Dive
- `ToTensor()` converts PIL image [0,255] → tensor [0,1]
- `Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])` converts [0,1] → [-1,1]
- **Formula:** `output = (input - mean) / std`
- **Why?** Centered data → stable gradients → faster convergence


## 📚 Learning Objectives

By the end of this notebook, you will:

1. ✅ Build `train_transform` with `transforms.Compose()`
2. ✅ Apply augmentations in the correct order
3. ✅ Understand which augmentations are realistic for histopathology
4. ✅ Normalize images to stabilize training
5. ✅ Verify transform output shape: `[3, 96, 96]`


## ✅ Acceptance Criteria

Your transform pipeline is correct when:

- [ ] `train_transform` is a `transforms.Compose` object
- [ ] Transforms are in order: Resize → RandomHorizontalFlip → RandomRotation → ColorJitter → ToTensor → Normalize
- [ ] Applying transform to a sample image produces a tensor of shape `[3, 96, 96]`
- [ ] Tensor values are in range `[-1, 1]` (after normalization)
- [ ] You can explain why ColorJitter comes **before** ToTensor


---

## 💻 TODO 1: Import Required Libraries

**What you need:**
- `torchvision.transforms` for transform classes
- `PIL.Image` to test loading a sample image

**Expected behavior:** Imports run without errors.


In [1]:
# TODO 1: Import transforms and Image
# Hint: from torchvision import transforms
# Hint: from PIL import Image

# YOUR CODE HERE
from torchvision import transforms
from PIL import Image


---

## 💻 TODO 2: Build the Training Transform Pipeline

**What you need to compose (in this order):**

1. **`transforms.Resize((96, 96))`** — Ensure all images are 96×96
2. **`transforms.RandomHorizontalFlip(p=0.5)`** — Flip left-right 50% of the time
3. **`transforms.RandomRotation(degrees=15)`** — Rotate ±15° randomly
4. **`transforms.ColorJitter(brightness=0.2, contrast=0.2)`** — Vary brightness/contrast
5. **`transforms.ToTensor()`** — Convert PIL image → tensor [0,1]
6. **`transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])`** — Normalize to [-1,1]

**Expected output:** A `transforms.Compose` object stored in `train_transform`.


In [3]:
# TODO 2: Create train_transform using transforms.Compose()
# Hint: train_transform = transforms.Compose([...])
# Hint: List the 6 transforms above in the correct order

# YOUR CODE HERE
train_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])  

print("✅ train_transform created:")
print(train_transform)


✅ train_transform created:
Compose(
    Resize(size=(96, 96), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)


---

## 💻 TODO 3: Test the Transform on a Sample Image

**What you need to do:**
1. Load a sample image from `../data/pcam_images/` (pick any `.png` file)
2. Apply `train_transform` to the image
3. Print the shape of the resulting tensor

**Expected output:**
```
Original image: PIL Image object
Transformed tensor shape: torch.Size([3, 96, 96])
Tensor value range: approximately [-1, 1]
```


In [8]:
# TODO 3: Test transform on a sample image
# Hint: sample_img = Image.open('../data/pcam_images/SOME_FILE.png')
# Hint: transformed = train_transform(sample_img)
# Hint: print(transformed.shape)

import os

# Get a sample image path
sample_images = os.listdir('../data/pcam_images/')
sample_path = os.path.join('../data/pcam_images/', sample_images[0])

# YOUR CODE HERE
# Load the image
with Image.open(sample_path) as sample_img:
    transformed = train_transform(sample_img)
# Apply train_transform
    train_transform
# Print the shape and value range
    print(transformed.shape)
    print("First value of shape refers to the number of channels and in this case is:", transformed.shape[0])
    print("Second and third values refer to the height and width of the image and in this case are:", transformed.shape[1], "and", transformed.shape[2])
    print("The value range is:", transformed.min(), "and", transformed.max())
print(f"✅ Sample image path: {sample_path}")


torch.Size([3, 96, 96])
First value of shape refers to the number of channels and in this case is: 3
Second and third values refer to the height and width of the image and in this case are: 96 and 96
The value range is: tensor(-0.8196) and tensor(1.)
✅ Sample image path: ../data/pcam_images/348.png


---

## 🤔 Reflection Prompts

### Question 1: Realistic Augmentations for Histopathology
Which of the following augmentations are **realistic** for H&E-stained pathology slides, and which might **distort clinically relevant information**?

| Augmentation | Realistic? | Reasoning |
|--------------|------------|-----------|
| RandomHorizontalFlip | ✅ / ❌ | ? |
| RandomVerticalFlip | ✅ / ❌ | ? |
| RandomRotation(±15°) | ✅ / ❌ | ? |
| RandomRotation(±180°) | ✅ / ❌ | ? |
| ColorJitter(brightness=0.2) | ✅ / ❌ | ? |
| ColorJitter(hue=0.5) | ✅ / ❌ | ? |
| RandomGrayscale | ✅ / ❌ | ? |

**Your analysis:**

---

### Question 2: Why Not Augment Validation/Test Data?
Explain in your own words:
- Why do we apply augmentation to training data?
- Why would augmentation **break** validation/test evaluation?

**Your explanation:**
> We augment training data to teach the model to handle natural variations (orientation, lighting, color shifts), preventing it from memorizing specific training examples. This helps generalization.
>
> We DON'T augment validation/test because we need **consistent, reproducible evaluation**. Each validation/test image should be processed the same way every time, so we can accurately measure model performance and track improvements.
>
> If we augmented validation data randomly each time, we'd get different results each run—making it impossible to know if the model improved or if we just got "lucky" augmented images that round!

---

### Question 3: Normalization Intuition
Given:
- `ToTensor()` converts RGB [0,255] → [0,1]
- `Normalize(mean=[0.5]*3, std=[0.5]*3)` converts [0,1] → [-1,1]

Calculate:
- If a pixel value is `0.8` after `ToTensor()`, what is it after `Normalize`?
- **Formula:** `(input - mean) / std`

**Your calculation:**
> Using the formula: `output = (input - mean) / std`
> 
> `output = (0.8 - 0.5) / 0.5`
> `output = 0.3 / 0.5`
> `output = 0.6`
>
> So a pixel value of `0.8` becomes `0.6` after normalization!
>
> **Visual meaning:** The pixel stays bright (80% → 60% of max), but now it's centered around 0 instead of 0.5. This helps stabilize training by keeping gradients centered.

---


## 🚀 Next Steps

Great work! You've built a training transform pipeline with augmentation.

**Move to Notebook 02:** Train Dataset & DataLoader

**Key Takeaway:** Transform order matters — Resize/Aug → ToTensor → Normalize!
